#IMPORTS

In [1]:
#Keras and TensorFlow
import keras
from keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
#Progress Bars
import time
import progressbar
#Encoding
from tensorflow.keras.preprocessing.sequence import pad_sequences
import sys
#Tokenize
import collections
from collections import Counter
import itertools
from itertools import chain
import string
import nltk as nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive

Mounted at /content/drive
/content/drive


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Dec 14 02:50:46 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


#Build Dataset

##Download Dataset

In [5]:
ds, info = tfds.load("billsum", with_info=True)
ds_ca_test, ds_test, ds_train = ds["ca_test"], ds["test"], ds["train"] 
print(info)
print(ds_train)

ds_numpy = tfds.as_numpy(ds_train);

#for summ in ds_numpy:
 # print(summ)

###PRINTS EVERY SUMMARY IN THE DS
#for ins in ds_train:
 # summary_i = ins["summary"]
  #print(summary_i)



#FeaturesDict({
 #   'summary': Text(shape=(), dtype=tf.string),
  #  'text': Text(shape=(), dtype=tf.string),
   # 'title': Text(shape=(), dtype=tf.string),
#})

Shuffling and writing examples to /root/tensorflow_datasets/billsum/3.0.0.incomplete5C6WGH/billsum-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/billsum/3.0.0.incomplete5C6WGH/billsum-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/billsum/3.0.0.incomplete5C6WGH/billsum-ca_test.tfrecord


Dataset billsum downloaded and prepared to /root/tensorflow_datasets/billsum/3.0.0. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='billsum',
    version=3.0.0,
    description='BillSum, summarization of US Congressional and California state bills.

There are several features:
  - text: bill text.
  - summary: summary of the bills.
  - title: title of the bills.
features for us bills. ca bills does not have.
  - text_len: number of chars in text.
  - sum_len: number of chars in summary.',
    homepage='https://github.com/FiscalNote/BillSum',
    features=FeaturesDict({
        'summary': Text(shape=(), dtype=tf.string),
        'text': Text(shape=(), dtype=tf.string),
        'title': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=23455,
    splits={
        'ca_test': 1237,
        'test': 3269,
        'train': 18949,
    },
    supervised_keys=('text', 'summary'),
    citation="""@misc{kornilova2019billsum,
        title={BillSum: A Corpus fo

##Dataset

####Clean DS Code from DS creator

In [6]:
import os
import pandas as pd
import pickle
import re
import unicodedata

def normalize_input(text):
  return unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
  
def replace_semicolon(text, threshold=10):
    '''
    Get rid of semicolons.
    First split text into fragments between the semicolons. If the fragment 
    is longer than the threshold, turn the semicolon into a period. O.w treat
    it as a comma.
    Returns new text
    '''
    new_text = ""
    for subset in re.split(';', text):
        subset = subset.strip() # Clear off spaces
        # Check word count
        if len(subset.split()) > threshold:
            # Turn first char into uppercase
            new_text += ". " + subset[0].upper() + subset[1:]
        else:
            # Just append with a comma 
            new_text += ", " + subset

    return new_text

USC_re = re.compile('[Uu]\.*[Ss]\.*[Cc]\.]+')
PAREN_re = re.compile('\([^(]+\ [^\(]+\)')
BAD_PUNCT_RE = re.compile(r'([%s])' % re.escape('"#%&\*\+/<=>@[\]^{|}~_'), re.UNICODE)
BULLET_RE = re.compile('\n[\ \t]*`*\([a-zA-Z0-9]*\)')
DASH_RE = re.compile('--+')
WHITESPACE_RE = re.compile('\s+')
EMPTY_SENT_RE = re.compile('[,\.]\ *[\.,]')
FIX_START_RE = re.compile('^[^A-Za-z]*')
FIX_PERIOD = re.compile('\.([A-Za-z])')
SECTION_HEADER_RE = re.compile('SECTION [0-9]{1,2}\.|\nSEC\.* [0-9]{1,2}\.|Sec\.* [0-9]{1,2}\.')

FIX_PERIOD = re.compile('\.([A-Za-z])')

SECTION_HEADER_RE = re.compile('SECTION [0-9]{1,2}\.|\nSEC\.* [0-9]{1,2}\.|Sec\.* [0-9]{1,2}\.')

def clean_text(text):
    """
    Borrowed from the FNDS text processing with additional logic added in.
    Note: we do not take care of token breaking - assume SPACY's tokenizer
    will handle this for us.
    """

    # Indicate section headers, we need them for features
    text = SECTION_HEADER_RE.sub('SECTION-HEADER', text)
    # For simplicity later, remove '.' from most common acronym
    text = text.replace("U.S.", "US")
    text = text.replace('SEC.', 'Section')
    text = text.replace('Sec.', 'Section')
    text = USC_re.sub('USC', text)

    # Remove parantheticals because they are almost always references to laws 
    # We could add a special tag, but we just remove for now
    # Note we dont get rid of nested parens because that is a complex re
    #text = PAREN_re.sub('LAWREF', text)
    text = PAREN_re.sub('', text)
    

    # Get rid of enums as bullets or ` as bullets
    text = BULLET_RE.sub(' ',text)
    
    # remove &nbsp
    text = text.replace('&nbsp', '')


    # Clean html 
    text = text.replace('&lt;all&gt;', '')

    # Remove annoying punctuation, that's not relevant
    text = BAD_PUNCT_RE.sub('', text)

    # Get rid of long sequences of dashes - these are formating
    text = DASH_RE.sub( ' ', text)

    # removing newlines, tabs, and extra spaces.
    text = WHITESPACE_RE.sub(' ', text)
    
    # If we ended up with "empty" sentences - get rid of them.
    text = EMPTY_SENT_RE.sub('.', text)
    
    # Attempt to create sentences from bullets 
    text = replace_semicolon(text)
    
    # Fix weird period issues + start of text weirdness
    #text = re.sub('\.(?=[A-Z])', '  . ', text)
    # Get rid of anything thats not a word from the start of the text
    text = FIX_START_RE.sub( '', text)
    # Sometimes periods get formatted weird, make sure there is a space between periods and start of sent   
    text = FIX_PERIOD.sub(". \g<1>", text)

    # Fix quotes
    text = text.replace('``', '"')
    text = text.replace('\'\'', '"')
    text = str(normalize_input(text))

    # Add special punct back in
    text = text.replace('SECTION-HEADER', '')
    text = text.replace('SHORT TITLE.', '')

    text = re.sub('Section [0-9]{1,2}\.', '', text)
    text = re.sub("(\w)quot(\s|\.|$)", r"\1 ", text)

    return text

###Tokenize

In [7]:
train_tokens_file = '/content/drive/My Drive/glove/train_tokens.pkl'
test_tokens_file =  '/content/drive/My Drive/glove/test_tokens.pkl'
import os.path
from os import path
#load object from file
def load_tok(datastruct_file):
  tok_set = None
  if path.exists(datastruct_file):
    dfile = open(datastruct_file, 'rb')
    tok_set = pickle.load(dfile)
    dfile.close()
  return tok_set

train_set = load_tok(train_tokens_file)
test_set = load_tok(test_tokens_file)

In [8]:
def clean_item(text):
  txt = (clean_text(re.sub("\(\d\)", '',(str(text)[12:-26]).replace("\\n", " ")))).lower()
  mytable = txt.maketrans(".-", "  ", ",'`;)\":")
  return txt.translate(mytable)

def clean_and_tokenize(datastruct):
  return word_tokenize(clean_item(datastruct['summary'])), word_tokenize(clean_item(datastruct['text']))

def format_data(datastruct):
  data_set = {'x': [], 'y': []}
  count = 0
  with progressbar.ProgressBar(max_value=len(datastruct)) as bar:
    for entry in datastruct:
      art, sum = clean_and_tokenize(entry)
      data_set['x'].append(art)
      data_set['y'].append(sum)
      count += 1
      bar.update(count)
  return data_set



In [9]:
#save data to file
def save(datastruct_file, dataset):
  dfile = open(datastruct_file, 'wb')
  pickle.dump(dataset, dfile)
  dfile.close()

if True:
  train_set = format_data(ds_train)
  test_set = format_data(ds_test)
  save(train_tokens_file, train_set)
  save(test_tokens_file, train_set)

100% (18949 of 18949) |##################| Elapsed Time: 0:02:43 Time:  0:02:43
100% (3269 of 3269) |####################| Elapsed Time: 0:00:28 Time:  0:00:28


In [10]:
def get_vocab(dataset):
    return Counter(item for subset in dataset for lst in dataset[subset] for item in lst).keys()

vocab_list = get_vocab(train_set)

###Embedding

####GloVe

In [11]:
#Creates Vectors
#taken from https://medium.com/analytics-vidhya/basics-of-using-pre-trained-glove-vectors-in-python-d38905f356db
vectors_dict = {}
glove_file_4B = '/content/drive/My Drive/glove/glove.6B.50d.txt'
glove_file_42B = '/content/drive/My Drive/glove/glove.42B.300d.txt'
size = 400000 #4B = 400000 , 42B = 1917494
embedding_dim = 50

with open(glove_file_4B, 'r') as GloVe:
  i = 0
  #with progressbar.ProgressBar(max_value=size) as bar:
  for line in GloVe:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], "float32")
    vectors_dict[word] = vector
    #  bar.update(i)
    i += 1
print(i)

400000


####Embedding *Matrix*

In [12]:
#Creates Embedding -> (word_index : vectors)
i = 0
failed = 0
embedding_dictionary = {}

with progressbar.ProgressBar(max_value=len(vocab_list)) as bar:
  for word in vocab_list:
    if word in vectors_dict:
      embedding_dictionary[word] = vectors_dict[word].astype('float32')
    else:
      failed += 1
      #print(word)
    bar.update(i)
    i += 1

word_count = len(embedding_dictionary)+2
print(f"Embeddings Dict size: {word_count}")
print(f"{failed} words did not match the glove")
#print(embedding_matrix[tokenizer.word_index["bureau"]])
print(sys.getsizeof(embedding_dictionary))

tokenizer = Tokenizer(num_words=word_count, oov_token='OOV')
tokenizer.fit_on_texts(embedding_dictionary.keys())

embedding_matrix = np.zeros((word_count, embedding_dim), dtype='float32')

for w in tokenizer.word_index:
  if w in embedding_dictionary:
    embedding_matrix[tokenizer.word_index[w]] = embedding_dictionary[w]

del embedding_dictionary

100% (74175 of 74175) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


Embeddings Dict size: 42053
32124 words did not match the glove
1310816


##New Encoding

In [13]:
max_article_length = 2000
max_summary_length = 2000
trunc_type='post'
padding_type='post'

def encode(data_set):
  art_sequences = tokenizer.texts_to_sequences(data_set['x'])
  art_padded = pad_sequences(art_sequences, maxlen=max_article_length, padding=padding_type, truncating=trunc_type)
  sum_sequences = tokenizer.texts_to_sequences(data_set['y'])
  sum_padded = pad_sequences(sum_sequences, maxlen=max_summary_length, padding=padding_type, truncating=trunc_type)
  return {'x': art_padded, 'y': sum_padded}

training_padded = encode(train_set)
testing_padded = encode(test_set)

del train_set
del test_set

###Decoding

In [14]:
def find_closest(embeddings, embedding_dict):
  divisors = np.matmul(np.linalg.norm(embeddings, axis=1).reshape(len(embeddings), 1), np.linalg.norm(embedding_dict, axis=1).reshape(1, len(embedding_dict)))+np.float32(0.00001)
  numerators = np.tensordot(embeddings, embedding_dict, axes=(1,1))
  return np.argmax(numerators/divisors, axis=1)

In [15]:
def decode(vocab_tokenizer, sequence_set, embedding_dict):
  sequences = [find_closest(item, embedding_dict) for item in sequence_set]
  return vocab_tokenizer.sequences_to_texts(sequences)

#Build Model Architecture

##Initialize


In [16]:
#Initializers

p_W, p_U, p_emb, p_dense, weight_decay = 0, 0, 0.1, 0, 0
rnn_layers, rnn_size = 5, 1024
from keras.regularizers import l2
regularizer = l2(weight_decay) if weight_decay else None


##Build Model

In [17]:
#Build Model
print(embedding_matrix.shape)
vocab_size, embedding_size = embedding_matrix.shape

model = keras.Sequential()
model.add(layers.Embedding(vocab_size, embedding_size, input_length=max_article_length,
                    embeddings_regularizer=regularizer, weights=[embedding_matrix], mask_zero=True))

for i in range(rnn_layers):
    lstm = layers.LSTM(rnn_size, return_sequences=True)
    model.add(lstm)
    model.add(layers.Dropout(p_dense))



model.add(layers.TimeDistributed(layers.Dense(embedding_dim)))
#model.add(Decode(embedding_matrix))
#model.add(layers.Reshape((max_summary_length, embedding_dim)))
model.add(layers.Activation('softmax'))

from keras.optimizers import RMSprop, Adam

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

(42053, 50)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2000, 50)          2102650   
_________________________________________________________________
lstm (LSTM)                  (None, 2000, 1024)        4403200   
_________________________________________________________________
dropout (Dropout)            (None, 2000, 1024)        0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 2000, 1024)        8392704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2000, 1024)        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2000, 1024)        8392704   
_________________________________________________________________
dropout_2 (Dropout)          (None, 2000, 10

#Build Input

##Train

#####Load Model

In [ ]:
model_path =  '/content/drive/My Drive/keras/models/test.keras'

model = keras.models.load(model_path)

In [19]:
def generator(dataset, batch_size, steps):
  idx=1
  while True: 
    start = (idx - 1) * batch_size
    yield (np.array(dataset['x'][start:start+batch_size]), np.array(embedding_matrix[dataset['y'][start:start+batch_size]]))
    if idx<steps:
      idx+=1
    else:
      idx=1

num_epochs = 10
batch_size = 16
steps_per_epoch = len(training_padded['x'])/batch_size
validation_steps = len(testing_padded['x'])/batch_size

train_gen = generator(training_padded, batch_size, steps_per_epoch)
test_gen = generator(testing_padded, batch_size, validation_steps)

model.fit(train_gen, epochs=num_epochs, steps_per_epoch=steps_per_epoch, 
  verbose=1, validation_data=test_gen, validation_steps=validation_steps)

Epoch 1/10
1185/1184 [==============================] - 626s 528ms/step - loss: -6.3317 - val_loss: -6.3338
Epoch 2/10
1185/1184 [==============================] - 620s 523ms/step - loss: -6.3816 - val_loss: -6.3380
Epoch 3/10
1185/1184 [==============================] - 618s 522ms/step - loss: -6.3886 - val_loss: -6.3423
Epoch 4/10
1185/1184 [==============================] - 618s 522ms/step - loss: -6.3947 - val_loss: -6.3522
Epoch 5/10
1185/1184 [==============================] - 618s 521ms/step - loss: -6.4059 - val_loss: -6.3640
Epoch 6/10
1185/1184 [==============================] - 618s 521ms/step - loss: -6.4401 - val_loss: -6.4135
Epoch 7/10
1185/1184 [==============================] - 618s 521ms/step - loss: -6.4732 - val_loss: -6.4327
Epoch 8/10
1185/1184 [==============================] - 617s 521ms/step - loss: -6.5020 - val_loss: -6.4640
Epoch 9/10
1185/1184 [==============================] - 617s 521ms/step - loss: -6.5405 - val_loss: -6.4976
Epoch 10/10
1185/1184 [=====

#####Save Model

In [ ]:
model_path =  '/content/drive/My Drive/keras/models/test.keras'

keras.models.save_model(model, model_path)

###Test Model

In [22]:
num_predictions = 3

test_set = np.asarray(testing_padded['x'][:num_predictions])
predictions = model.predict(test_set)

decoded_predictions = decode(tokenizer, predictions, embedding_matrix)
print(sequence for sequence in decoded_predictions)

<generator object <genexpr> at 0x7f5a344e5f68>
